In [1]:
#Imports
import sklearn as sk
import pandas as pd 
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import tweepy as tw
from decouple import config
import json
import tweepy
from tweepy import Stream                   
from tweepy.streaming import StreamListener 

In [2]:
#Login Tweepy Credenciales Twitter
consumer_key = 'add-your-tokens-here'
consumer_secret = 'add-your-tokens-here'
access_token = 'add-your-tokens-here'
access_token_secret = 'add-your-tokens-here'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
def get_tuits(userID):
    print("Entré con "+userID)
    all_tweets = []
    oldest_id = 0
    while True:
        print("Entré al while")
        if oldest_id == 0:
            tweets = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False,
                           tweet_mode = 'extended'
                           )
        else: 
              tweets = api.user_timeline(screen_name=userID, 
                                   count=200,
                                   include_rts = False,
                                   max_id = oldest_id - 1,
                                   tweet_mode = 'extended'
                                   )
        if len(tweets) == 0:
            print("len(tweets) == 0. Como no puedo extraer más nada me voy")
            break
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)
        print('Número de tweets bajados hasta el momento {}'.format(len(all_tweets)))
    return all_tweets

In [4]:
def create_dataframe(tuits, user):
    outtweets = [[tweet.id_str, 
                  tweet.created_at, 
                  tweet.favorite_count, 
                  tweet.retweet_count, 
                  tweet.full_text.encode("utf-8").decode("utf-8")] 
                 for idx,tweet in enumerate(tuits)]
    df = pd.DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
    df.to_csv('%s_tweets.csv' % user,index=False)
    df.head(3)
    return df

In [5]:
def createPopularTuitChart(df):
    ylabels = ["favorite_count","retweet_count"]

    fig = plt.figure(figsize=(13,3))
    fig.subplots_adjust(hspace=0.01,wspace=0.01)

    n_row = len(ylabels)
    n_col = 1
    for count, ylabel in enumerate(ylabels):
        ax = fig.add_subplot(n_row,n_col,count+1)
        ax.plot(df["created_at"],df[ylabel])
        ax.set_ylabel(ylabel)
    plt.show()

In [6]:
def print_most_popular_tuits(df, max_fav, max_retweet):
    df_sub = df.loc[(df["favorite_count"] > max_fav) | (df["retweet_count"] > max_retweet),:]
    for irow in range(df_sub.shape[0]):
        df_row = df_sub.iloc[irow,:]

        print(df_row["created_at"])
        print("favorite_count={:6} retweet_count={:6}".format(df_row["favorite_count"],df_row["retweet_count"]))
        print(df_row["text"])
        print("\n")

In [7]:
search_words = 'Ukraine'
date_since = ''
date_until = ''
ntweets = 1000
location= ''
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(ntweets)

lista = [[tweet.text, tweet.id, tweet.entities, tweet.created_at] for tweet in tweets]

In [8]:
df = pd.DataFrame(lista, columns=['Text','id','entities','created_at'])
df = df.sort_values(by=['created_at'], ascending=True)
df

,Text,id,entities,created_at
999,"RT @IAPonomarenko: Russians, of course, will n...",1507336609095733250,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:40:27
987,"RT @FortniteGame: As of today, we’ve raised $5...",1507336611792662530,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:40:27
988,RT @spriter99880: Russia’s Federal Security Se...,1507336611599577089,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:40:27
989,"RT @McFaul: ""Ukrainians will ultimately defeat...",1507336611008331781,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:40:27
990,RT @UAWeapons: #Ukraine: Claimed video of a Uk...,1507336610903379978,"{'hashtags': [{'text': 'Ukraine', 'indices': [...",2022-03-25 12:40:27
...,...,...,...,...
25,RT @APompliano: Bitcoin is up nearly 30% since...,1507336851673264130,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:41:25
14,RT @POTUS: It was an honor to meet with Europe...,1507336853879508995,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:41:25
2,RT @spectatorindex: BREAKING: Ukraine's Defens...,1507336855700013058,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:41:26
1,RT @caitoz: The US alliance is pouring billion...,1507336856123359233,"{'hashtags': [], 'symbols': [], 'user_mentions...",2022-03-25 12:41:26


In [9]:
#Control tweets are unique
len(set(df['id'])) 

1000

In [10]:
df = pd.DataFrame(lista)
df.drop(columns=[1,2,3],axis = 1, inplace=True)

In [11]:
df

,0
0,RT @KyivIndependent: Ukraine's General Staff r...
1,RT @caitoz: The US alliance is pouring billion...
2,RT @spectatorindex: BREAKING: Ukraine's Defens...
3,RT @ALocalAmerican: @TexitDallasCo Remember. B...
4,RT @dkaleniuk: Brussels yesterday right near t...
...,...
995,@dtimes17 😷/ What price do we need to pay to c...
996,"@Mswahili_ke Huko hakunanga wamama,people ther..."
997,Make the World Cup 33 teams. A group of five. ...
998,"RT @neoggumdol: @KyivIndependent ""About 1,300 ..."


In [12]:
df.to_csv('tweets_ukraine.csv')